# Solving Laplace's Equation on an Annulus using PINNs with DeepXDE\n\nThis notebook demonstrates how to use Physics-Informed Neural Networks (PINNs) with DeepXDE to solve Laplace's equation (Δu = 0) on an annular domain.\n\n**Laplace's Equation:** Δu(x, y) = 0\n\n**Annulus Domain:** The domain is an annulus centered at the origin [0,0] with an inner radius of 1 and an outer radius of 2.\n\n**Boundary Conditions:**\n1. Outer boundary (r=2): u = 0 (Dirichlet)\n2. Inner boundary (r=1): u = sin(θ) (Dirichlet, angle-dependent)

## 1. Setup: Import Libraries

In [ ]:
import deepxde as dde\nimport numpy as np\nimport matplotlib.pyplot as plt

## 2. Geometry Definition\n\nThe annulus is defined by an outer disk (radius 2) and an inner disk (radius 1), both centered at the origin. The annulus is the region between these two circles.

In [ ]:
center = [0, 0]\nr_outer = 2.0\nr_inner = 1.0\n\ndisk_outer = dde.geometry.Disk(center, r_outer)\ndisk_inner = dde.geometry.Disk(center, r_inner)\n\n# Define the annulus using CSG (Constructive Solid Geometry) difference\ngeom = disk_outer - disk_inner\n\n# Optional: Plot geometry to verify\nfig, ax = plt.subplots(figsize=(6,6))\nax.add_patch(plt.Circle(center, r_outer, color='lightblue', alpha=0.7, label='Outer Disk Region'))\nax.add_patch(plt.Circle(center, r_inner, color='white', label='Inner Disk Region (Removed)'))\nax.plot(*dde.geometry.Disk(center, r_outer).boundary_points().T, 'b-', label='Outer Boundary (r=2)')\nax.plot(*dde.geometry.Disk(center, r_inner).boundary_points().T, 'r-', label='Inner Boundary (r=1)')\nax.set_xlim([-r_outer-0.5, r_outer+0.5])\nax.set_ylim([-r_outer-0.5, r_outer+0.5])\nax.set_aspect('equal', adjustable='box')\nplt.xlabel('x')\nplt.ylabel('y')\nplt.title('Annulus Domain Geometry')\nplt.legend()\nplt.grid(True)\nplt.show()

## 3. PDE and Boundary Condition Definition\n\n**PDE:** Δu = 0 (Laplacian of u is zero)\n\n**Boundary Conditions:**\n- Outer (r=2): u = 0\n- Inner (r=1): u = sin(θ)

In [ ]:
def pde(x, y_pred):\n    """Defines Laplace's equation: dy_xx + dy_yy = 0"""\n    dy_xx = dde.grad.hessian(y_pred, x, i=0, j=0)\n    dy_yy = dde.grad.hessian(y_pred, x, i=1, j=1)\n    return dy_xx + dy_yy\n\n# Outer Boundary (r=2, u=0)\ndef boundary_outer(x_coords, on_boundary_annulus):\n    return np.isclose(np.linalg.norm(x_coords, axis=-1), r_outer)\n\ndef func_outer_val(x_coords):\n    return np.zeros((x_coords.shape[0], 1), dtype=dde.config.real(np))\n\nbc_outer = dde.DirichletBC(geom, func_outer_val, boundary_outer)\n\n# Inner Boundary (r=1, u=sin(theta))\ndef boundary_inner(x_coords, on_boundary_annulus):\n    return np.isclose(np.linalg.norm(x_coords, axis=-1), r_inner)\n\ndef func_inner_val(x_coords):\n    cart_x = x_coords[:, 0:1]\n    cart_y = x_coords[:, 1:2]\n    theta = np.arctan2(cart_y, cart_x)\n    return np.sin(theta)\n\nbc_inner = dde.DirichletBC(geom, func_inner_val, boundary_inner)\n\nbcs = [bc_outer, bc_inner]

## 4. Problem Setup (Data)

In [ ]:
N_domain = 3000\nN_boundary = 1500\nN_test = 1000\ndata = dde.data.PDE(\n    geom,\n    pde,\n    bcs,\n    num_domain=N_domain,\n    num_boundary=N_boundary,\n    num_test=N_test,\n    solution=None\n)

## 5. Network and Model Creation

In [ ]:
input_dim = 2\noutput_dim = 1\nnum_hidden_layers = 4\nneurons_per_layer = 64\nactivation_fn = "tanh"\nkernel_initializer = "Glorot uniform"\nnet = dde.maps.FNN(\n    [input_dim] + [neurons_per_layer] * num_hidden_layers + [output_dim],\n    activation_fn,\n    kernel_initializer\n)\nmodel = dde.Model(data, net)

## 6. Model Training

In [ ]:
learning_rate = 1e-3\nmodel.compile("adam", lr=learning_rate)\nnum_iterations = 25000\nprint(f"Starting training for {num_iterations} iterations...")\nlosshistory, train_state = model.train(iterations=num_iterations, display_every=1000)\nprint("Training finished.")

## 7. Results and Visualization

In [ ]:
dde.utils.plot_loss_history(losshistory)\nplt.title('Loss History for Annulus Problem (Log Scale)')\nplt.show()\nplot_resolution = 100\nx_lin = np.linspace(-r_outer, r_outer, plot_resolution)\ny_lin = np.linspace(-r_outer, r_outer, plot_resolution)\nX_plot, Y_plot = np.meshgrid(x_lin, y_lin)\nplot_points_flat = np.vstack((X_plot.ravel(), Y_plot.ravel())).T\nis_inside_domain = geom.inside(plot_points_flat)\nplot_points_inside_annulus = plot_points_flat[is_inside_domain]\nif plot_points_inside_annulus.shape[0] > 0:\n    u_pred_annulus = model.predict(plot_points_inside_annulus)\n    plt.figure(figsize=(8, 7))\n    contour = plt.tricontourf(\n        plot_points_inside_annulus[:, 0],\n        plot_points_inside_annulus[:, 1],\n        u_pred_annulus.ravel(),\n        levels=100,\n        cmap='viridis'\n    )\n    plt.colorbar(contour, label='Predicted u(x,y)')\n    plt.xlabel('x')\n    plt.ylabel('y')\n    plt.title('Predicted Solution u(x,y) on Annulus Domain')\n    ax = plt.gca()\n    ax.set_aspect('equal', adjustable='box')\n    circle_inner_plot = plt.Circle(center, r_inner, color='black', fill=False, linestyle='--', linewidth=1.0)\n    circle_outer_plot = plt.Circle(center, r_outer, color='black', fill=False, linestyle='--', linewidth=1.0)\n    ax.add_artist(circle_inner_plot)\n    ax.add_artist(circle_outer_plot)\nelse:\n    plt.figure(figsize=(8, 7))\n    plt.text(0.5, 0.5, \"No points found inside annulus for plotting\", \n             horizontalalignment='center', verticalalignment='center', \n             transform=plt.gca().transAxes)\n    plt.xlabel('x')\n    plt.ylabel('y')\n    plt.title('Predicted Solution u(x,y) on Annulus Domain')\nplt.show()

## 8. Analytical Solution for Annulus Problem\n\nFor the specific problem of Laplace's equation (Δu = 0) on an annulus (inner radius r_i=1, outer radius r_o=2) with boundary conditions u(r_i, θ) = sin(θ) and u(r_o, θ) = 0, the analytical solution can be derived using separation of variables in polar coordinates.\n\nThe general solution for Laplace's equation in polar coordinates that is periodic in θ is of the form:\nu(r, θ) = (A_0 + B_0 ln r) + Σ (A_n r^n + B_n r^-n) cos(nθ) + Σ (C_n r^n + D_n r^-n) sin(nθ)\n\nGiven our specific boundary conditions:\n1. u(r=2, θ) = 0\n2. u(r=1, θ) = sin(θ)\n\nOnly the sin(nθ) terms with n=1 will be non-zero. So the solution form simplifies to u(r, θ) = (C_1 r + D_1 r^-1)sin(θ).\nApplying the boundary conditions:\n- At r=2: (2*C_1 + D_1/2)sin(θ) = 0  =>  2*C_1 + D_1/2 = 0\n- At r=1: (C_1*1 + D_1/1)sin(θ) = sin(θ) => C_1 + D_1 = 1\n\nFrom C_1 + D_1 = 1, we get D_1 = 1 - C_1.\nSubstitute into the first equation: 2*C_1 + (1 - C_1)/2 = 0  =>  4*C_1 + 1 - C_1 = 0  =>  3*C_1 = -1  =>  C_1 = -1/3.\nThen D_1 = 1 - (-1/3) = 4/3.\n\nSo, the analytical solution is: **u(r, θ) = (-1/3 * r + 4/3 * r^-1)sin(θ)**.

In [ ]:
def u_analytical_cartesian(coords, r_inner_an, r_outer_an):\n    x_c = coords[:, 0]\n    y_c = coords[:, 1]\n    r = np.sqrt(x_c**2 + y_c**2)\n    theta = np.arctan2(y_c, x_c)\n    C1 = -1/3\n    D1 = 4/3\n    return (C1 * r + D1 / r) * np.sin(theta)\nif 'plot_points_inside_annulus' not in locals() or plot_points_inside_annulus.shape[0] == 0:\n    print("Regenerating points for analytical plot...")\n    _plot_resolution = 100\n    _x_lin = np.linspace(-r_outer, r_outer, _plot_resolution)\n    _y_lin = np.linspace(-r_outer, r_outer, _plot_resolution)\n    _X_plot, _Y_plot = np.meshgrid(_x_lin, _y_lin)\n    _plot_points_flat = np.vstack((_X_plot.ravel(), _Y_plot.ravel())).T\n    _is_inside_domain = geom.inside(_plot_points_flat)\n    plot_points_inside_annulus = _plot_points_flat[_is_inside_domain]\nif plot_points_inside_annulus.shape[0] > 0:\n    u_analytical_values = u_analytical_cartesian(plot_points_inside_annulus, r_inner, r_outer)\n    plt.figure(figsize=(8, 7))\n    contour_analytical = plt.tricontourf(\n        plot_points_inside_annulus[:, 0],\n        plot_points_inside_annulus[:, 1],\n        u_analytical_values.ravel(),\n        levels=100,\n        cmap='viridis'\n    )\n    plt.colorbar(contour_analytical, label='Analytical u(x,y)')\n    plt.xlabel('x')\n    plt.ylabel('y')\n    plt.title('Analytical Solution on Annulus Domain')\n    ax_an = plt.gca()\n    ax_an.set_aspect('equal', adjustable='box')\n    circle_inner_plot_an = plt.Circle(center, r_inner, color='black', fill=False, linestyle='--', linewidth=1.0)\n    circle_outer_plot_an = plt.Circle(center, r_outer, color='black', fill=False, linestyle='--', linewidth=1.0)\n    ax_an.add_artist(circle_inner_plot_an)\n    ax_an.add_artist(circle_outer_plot_an)\n    plt.show()\nelse:\n    print("No points found inside annulus to plot analytical solution.")

## 9. Finite Element Method (FEM) Solution with FEniCSx\n\nAs a comparison, we can also solve Laplace's equation on the annulus using the Finite Element Method (FEM). Below is an example using the FEniCSx library. \n\n**Note:** Running the following cells requires a working installation of FEniCSx (including `dolfinx` and `gmsh` for meshing, or an alternative meshing strategy if `gmsh` is not used directly via Python API). If FEniCSx is not installed, these cells will raise an error.

In [ ]:
try:\n    import dolfinx\n    import dolfinx.fem as fem\n    import dolfinx.mesh as dmesh\n    from dolfinx.fem.petsc import LinearProblem\n    import ufl\n    from mpi4py import MPI\n    import gmsh\n    fenicsx_available = True\n    print(f\"FEniCSx version: {dolfinx.__version__}\")\nexcept ImportError:\n    fenicsx_available = False\n    print(\"FEniCSx is not installed. Skipping FEM solution cells.\")\nif fenicsx_available:\n    comm = MPI.COMM_WORLD\n    model_rank = 0

In [ ]:
if fenicsx_available:\n    mesh_comm = comm\n    gdim = 2\n    gmsh.initialize()\n    if comm.rank == model_rank:\n        gmsh.model.add(\"annulus_fem\")\n        p_center = gmsh.model.occ.addPoint(center[0], center[1], 0)\n        c_outer = gmsh.model.occ.addCircle(center[0], center[1], 0, r_outer)\n        c_inner = gmsh.model.occ.addCircle(center[0], center[1], 0, r_inner)\n        cl_outer = gmsh.model.occ.addCurveLoop([c_outer])\n        cl_inner = gmsh.model.occ.addCurveLoop([c_inner])\n        s_annulus = gmsh.model.occ.addPlaneSurface([cl_outer, cl_inner])\n        gmsh.model.occ.synchronize()\n        gmsh.model.addPhysicalGroup(1, [c_inner], tag=1)\n        gmsh.model.addPhysicalGroup(1, [c_outer], tag=2)\n        gmsh.model.addPhysicalGroup(2, [s_annulus], tag=3)\n        gmsh.option.setNumber(\"Mesh.CharacteristicLengthMin\", 0.1)\n        gmsh.option.setNumber(\"Mesh.CharacteristicLengthMax\", 0.1)\n        gmsh.model.mesh.generate(gdim)\n    domain, cell_tags, facet_tags = dolfinx.io.gmshio.model_to_mesh(gmsh.model, comm, model_rank, gdim=gdim)\n    gmsh.finalize()\n    tdim = domain.topology.dim\n    fdim = tdim - 1

In [ ]:
if fenicsx_available:\n    V = fem.FunctionSpace(domain, ("Lagrange", 1))\n    outer_facets = facet_tags.find(2)\n    u_outer_val = fem.Constant(domain, dolfinx.default_scalar_type(0.0))\n    bc_outer_fem = fem.dirichletbc(u_outer_val, fem.locate_dofs_topological(V, fdim, outer_facets), V)\n    inner_facets = facet_tags.find(1)\n    u_inner_expr = fem.Function(V)\n    def sin_theta_func(x_coords_fem):\n        theta_fem = np.arctan2(x_coords_fem[1], x_coords_fem[0])\n        return np.sin(theta_fem)\n    u_inner_expr.interpolate(sin_theta_func)\n    bc_inner_fem = fem.dirichletbc(u_inner_expr, fem.locate_dofs_topological(V, fdim, inner_facets), V)\n    bcs_fem = [bc_inner_fem, bc_outer_fem]

In [ ]:
if fenicsx_available:\n    u = ufl.TrialFunction(V)\n    v = ufl.TestFunction(V)\n    f_source = fem.Constant(domain, dolfinx.default_scalar_type(0.0))\n    a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx\n    L = f_source * v * ufl.dx\n    problem = LinearProblem(a, L, bcs=bcs_fem, petsc_options={\"ksp_type\": \"preonly\", \"pc_type\": \"lu\"})\n    uh_fem = problem.solve()\n    uh_fem.name = \"u_FEM\"

In [ ]:
if fenicsx_available:\n    if 'plot_points_inside_annulus' in locals() and plot_points_inside_annulus.shape[0] > 0:\n        points_for_eval = np.zeros((3, plot_points_inside_annulus.shape[0]))\n        points_for_eval[:gdim, :] = plot_points_inside_annulus.T\n        bb_tree = dolfinx.geometry.BoundingBoxTree(domain, domain.topology.dim)\n        cell_candidates = dolfinx.geometry.compute_collisions_points(bb_tree, points_for_eval)\n        colliding_cells = dolfinx.geometry.compute_colliding_cells(domain, cell_candidates, points_for_eval)\n        u_fem_on_grid = np.full(plot_points_inside_annulus.shape[0], np.nan)\n        points_on_proc_fem = []\n        cells_fem_plot = []\n        original_indices_plot = []\n        for i, point in enumerate(points_for_eval.T):\n            if len(colliding_cells.links(i)) > 0:\n                points_on_proc_fem.append(point)\n                cells_fem_plot.append(colliding_cells.links(i)[0])\n                original_indices_plot.append(i)\n        if points_on_proc_fem:\n            u_fem_values_at_points = uh_fem.eval(np.array(points_on_proc_fem).T, cells_fem_plot)\n            for idx_map, val in zip(original_indices_plot, u_fem_values_at_points):\n                u_fem_on_grid[idx_map] = val\n            valid_fem_plot_indices = ~np.isnan(u_fem_on_grid)\n            plt.figure(figsize=(8, 7))\n            contour_fem = plt.tricontourf(\n                plot_points_inside_annulus[valid_fem_plot_indices, 0],\n                plot_points_inside_annulus[valid_fem_plot_indices, 1],\n                u_fem_on_grid[valid_fem_plot_indices].ravel(),\n                levels=100,\n                cmap='viridis'\n            )\n            plt.colorbar(contour_fem, label='FEM u(x,y)')\n            plt.xlabel('x')\n            plt.ylabel('y')\n            plt.title('FEM Solution on Annulus Domain (on evaluation points)')\n            ax_fem = plt.gca()\n            ax_fem.set_aspect('equal', adjustable='box')\n            circle_i = plt.Circle(center, r_inner, color='black', fill=False, linestyle='--', linewidth=1.0)\n            circle_o = plt.Circle(center, r_outer, color='black', fill=False, linestyle='--', linewidth=1.0)\n            ax_fem.add_artist(circle_i)\n            ax_fem.add_artist(circle_o)\n            plt.show()\n        else:\n            print(\"FEM: No evaluation points found on the current MPI rank or in cells for plotting.\")\n    else:\n        print(\"FEM Plot: plot_points_inside_annulus not available or empty.\")

## 10. Comparison of Solutions\n\nTo quantitatively compare the PINN and FEM solutions with the analytical solution, we can compute the L2 relative error on the set of plotting points within the annulus.

In [ ]:
error_pinn = np.nan\nerror_fem = np.nan\nif 'plot_points_inside_annulus' in locals() and plot_points_inside_annulus.shape[0] > 0:\n    if 'u_analytical_values' in locals() and u_analytical_values is not None:\n        if 'u_pred_annulus' in locals() and u_pred_annulus is not None:\n            l2_diff_pinn = np.linalg.norm(u_pred_annulus.ravel() - u_analytical_values.ravel())\n            l2_norm_analytical = np.linalg.norm(u_analytical_values.ravel())\n            if l2_norm_analytical > 1e-8:\n                error_pinn = l2_diff_pinn / l2_norm_analytical\n                print(f\"L2 relative error (PINN vs Analytical): {error_pinn:.4e}\")\n            else:\n                print(\"L2 norm of analytical solution is very small; skipping PINN error calculation.\")\n        else:\n            print(\"PINN solution (u_pred_annulus) not found for error calculation.\")\n        if fenicsx_available and 'uh_fem' in locals() and 'u_fem_on_grid' in locals():\n            valid_fem_indices = ~np.isnan(u_fem_on_grid)\n            if np.any(valid_fem_indices):\n                l2_diff_fem = np.linalg.norm(u_fem_on_grid[valid_fem_indices].ravel() - u_analytical_values[valid_fem_indices].ravel())\n                l2_norm_analytical_subset = np.linalg.norm(u_analytical_values[valid_fem_indices].ravel())\n                if l2_norm_analytical_subset > 1e-8:\n                    error_fem = l2_diff_fem / l2_norm_analytical_subset\n                    print(f\"L2 relative error (FEM vs Analytical, on {np.sum(valid_fem_indices)} points): {error_fem:.4e}\")\n                else:\n                    print(\"L2 norm of analytical solution (subset) is very small; skipping FEM error calculation.\")\n            else:\n                print(\"Could not evaluate FEM solution on any of the common grid points for error calc.\")\n        elif not fenicsx_available:\n            print(\"FEM solution not computed (FEniCSx not available), skipping FEM error.\")\n        else:\n            print(\"FEM solution (uh_fem or u_fem_on_grid) not found for error calculation.\")\n    else:\n        print(\"Analytical solution (u_analytical_values) not found for error calculation.\")\nelse:\n    print(\"Common plotting points (plot_points_inside_annulus) not found for error calculation.\")

## 11. Updated Conclusion\n\nThis notebook demonstrated solving Laplace's equation on an annulus with angle-dependent Dirichlet boundary conditions using DeepXDE (a PINN approach).\n\nKey steps for the PINN solution included:\n1. Defining the annulus geometry using CSG operations.\n2. Setting up Laplace's PDE and two distinct Dirichlet boundary conditions.\n3. Configuring the PINN model (network architecture, data sampling).\n4. Training the model.\n5. Visualizing the loss history and the 2D solution plot.\n\nAdditionally, the notebook was extended to include:\n1. **Analytical Solution:** Derivation, implementation, and visualization of the exact analytical solution for this specific problem: `u(r,θ) = (-1/3 * r + 4/3 / r)sin(θ)`.\n2. **FEM Solution:** An example implementation using FEniCSx to solve the same problem using the Finite Element Method, including mesh generation, boundary condition application, solving, and visualization.\n3. **Comparison:** A quantitative comparison of L2 relative errors between the PINN solution and the analytical solution, and between the FEM solution and the analytical solution, on a common grid of points.\n\nThis multi-faceted approach allows for a richer understanding and validation of the PINN solution against both theoretical results and established numerical methods like FEM.\n\nFurther explorations could involve:\n- Varying the complexity of boundary conditions or the PDE.\n- Investigating the impact of network architecture, number of training points, and training iterations on PINN accuracy.\n- Exploring different FEM parameters (e.g., mesh density, element order) and their effect on FEM accuracy.\n- Applying these methods to 3D problems or time-dependent PDEs.